In [1]:
from dataclasses import dataclass
from enum import Enum
from collections import namedtuple


class TokenType(Enum):
    LeftParen = 1
    RightParen = 2
    Number = 3
    Plus = 4
    Star = 5

    
Token = namedtuple("Token", "token_type,token_value")
       
    
def tokenize(line):
    """Breaks homework line into token types,
    matching parens as we go
    """
    result = []
    paren_counter = 0
    for ch in line:
        if ch.isspace():
            continue
        elif ch.isnumeric():
            result.append(Token(TokenType.Number, int(ch)))
        elif ch == '(':
            result.append(Token(TokenType.LeftParen, paren_counter))
            paren_counter += 1
        elif ch == ')':
            paren_counter -= 1
            result.append(Token(TokenType.RightParen, paren_counter))
        elif ch == '+':
            result.append(Token(TokenType.Plus, 0))
        elif ch == '*':
            result.append(Token(TokenType.Star, 0))
        else:
            raise ValueError("Unexpected Character")
    return result
   

def next_value(tokenlist, evaluator):
    """Pull a number off the tokenlist, either by returning the 
    first item, if it's a number, or by evaluating the first
    expression if the first item is a left-paren
    
    evaluator is an arg so that can switch between problem 1
    and problem 2 evaluation rules
    
    Returns: number and list of unconsumed tokens"""
    
    token = tokenlist[0]
    
    if token.token_type == TokenType.Number:
        return token.token_value, tokenlist[1:]
        
    elif token.token_type == TokenType.LeftParen:
        
        # evaluate expresson from left-paren to matching
        # right paren
        for index, seek_token in enumerate(tokenlist):
            if ((seek_token.token_type == TokenType.RightParen) and 
                (seek_token.token_value == token.token_value)):
                
                value = evaluator(tokenlist[1:index])
                return value, tokenlist[index + 1:]
            
    else:
        raise ValueError("Unexpected Token")
       

# Problem 1

The homework (your puzzle input) consists of a series of expressions that consist of addition (+), multiplication (*), and parentheses ((...)). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

In [2]:
def evaluate(tokenlist): 
    value, tokenlist = next_value(tokenlist, evaluate)
    
    while len(tokenlist):
        operation = tokenlist[0]
        value2, tokenlist = next_value(tokenlist[1:], evaluate)
    
        if operation.token_type == TokenType.Plus:
            value =  value + value2
        elif operation.token_type == TokenType.Star:
            value = value * value2
            
    return value

In [3]:
test = '1 + 2 * 3 + 4 * 5 + 6'    
tokens = tokenize(test)
evaluate(tokens)

71

In [4]:
test = '1 + (2 * 3) + (4 * (5 + 6))'
tokens = tokenize(test)
evaluate(tokens)

51

In [5]:
with open('homework.txt') as fp:
    data = fp.read()
    
total = 0
for line in data.splitlines():
    total += evaluate(tokenize(line))
total

75592527415659

# Problem 2

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication.

In [6]:
def evaluate2(tokenlist):
    
    if not len(tokenlist):
        raise ValueError("Empty Tokenlist")
        
    value, tokenlist = next_value(tokenlist, evaluate2)
    
    while len(tokenlist):
        operation = tokenlist[0]
        tokenlist = tokenlist[1:]
        
        if operation.token_type == TokenType.Plus:
            value2, tokenlist = next_value(tokenlist, evaluate2)
            value = value + value2
            
        elif operation.token_type == TokenType.Star:
            value2 = evaluate2(tokenlist)
            value = value * value2
            return value
        
        else:
            raise ValueError("Unexpected token")
            
    return value
        

In [7]:
test = '1 + 2 * 3 + 4 * 5 + 6'    
tokens = tokenize(test)

evaluate2(tokens)

231

In [8]:
test = '1 + (2 * 3) + (4 * (5 + 6))'
tokens = tokenize(test)
evaluate2(tokens)

51

In [9]:
with open('homework.txt') as fp:
    data = fp.read()
    
total = 0
for line in data.splitlines():
    total += evaluate2(tokenize(line))
total

360029542265462